# Task 4 — Business Optimization using Linear Programming (PuLP)

**Objective:** Solve a production planning problem for a factory that produces Product A and Product B.
This notebook demonstrates problem statement, mathematical formulation, PuLP implementation, results, visualization,
and interpretation / business insights.

---

## Problem Statement

A factory produces **Product A** and **Product B**.

- Profit per unit:
  - Product A → \$20
  - Product B → \$30

- Resource requirements per unit:
  - Product A → 2 hours machine, 3 hours labor
  - Product B → 4 hours machine, 2 hours labor

- Resource capacities:
  - Machine time ≤ 100 hours
  - Labor time ≤ 90 hours

**Goal:** Determine how many units of A and B to produce to **maximize total profit**.

## Mathematical Formulation

Let:
- xA = units of Product A
- xB = units of Product B

Objective:
Maximize: 20·xA + 30·xB

Subject to:
- 2·xA + 4·xB ≤ 100   (machine time)
- 3·xA + 2·xB ≤ 90    (labor time)
- xA, xB ≥ 0

In [ ]:
# Implementation using PuLP
# If PuLP is not installed in your environment, uncomment and run:
# !pip install pulp matplotlib

import pulp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

# Ensure plots show inline (Jupyter)
%matplotlib inline

In [ ]:
# Define the LP problem (maximize)
model = pulp.LpProblem("Factory_Optimization", pulp.LpMaximize)

# Decision variables
xA = pulp.LpVariable("Product_A", lowBound=0, cat="Continuous")
xB = pulp.LpVariable("Product_B", lowBound=0, cat="Continuous")

# Objective function
model += 20 * xA + 30 * xB, "Total_Profit"

# Constraints
model += 2 * xA + 4 * xB <= 100, "Machine_Time"
model += 3 * xA + 2 * xB <= 90, "Labor_Time"

# Solve (CBC solver)
model.solve(pulp.PULP_CBC_CMD(msg=False))

# Results
status = pulp.LpStatus[model.status]
xA_opt = pulp.value(xA)
xB_opt = pulp.value(xB)
profit_opt = pulp.value(model.objective)

print("Status:", status)
print("Optimal Product A:", xA_opt)
print("Optimal Product B:", xB_opt)
print("Maximum Profit = $", profit_opt)

In [ ]:
# Visualize feasible region and optimal solution

x = np.linspace(0, 50, 400)

# Constraint boundary functions (solved for xB)
y_machine = (100 - 2 * x) / 4    # from machine constraint
y_labor = (90 - 3 * x) / 2       # from labor constraint

# Clip negatives to zero for display
y_machine_clip = np.maximum(y_machine, 0)
y_labor_clip = np.maximum(y_labor, 0)

plt.figure(figsize=(9, 6))
plt.plot(x, y_machine_clip, label="Machine: 2xA + 4xB = 100")
plt.plot(x, y_labor_clip, label="Labor: 3xA + 2xB = 90")
plt.fill_between(x, np.minimum(y_machine_clip, y_labor_clip), 0, alpha=0.25, color="lightblue")

# Optimal point
plt.plot(xA_opt, xB_opt, "ro", markersize=8, label=f"Optimal ({xA_opt:.2f}, {xB_opt:.2f})")

plt.xlim(0, max(30, xA_opt + 5))
plt.ylim(0, max(30, xB_opt + 5))
plt.xlabel("Product A (units)")
plt.ylabel("Product B (units)")
plt.title("Feasible Region and Optimal Production Plan")
plt.legend()
plt.grid(True)

# Save figure (optional)
plt.tight_layout()
plt.savefig("feasible_region.png", dpi=150)
plt.show()

## Solution & Interpretation

- **Status:** Optimal
- **Optimal production plan:**
  - Product A = **15.0** units
  - Product B = **17.5** units
- **Maximum profit:** **$825.0**

**Interpretation:**  
The LP solver finds a corner (boundary) solution where the machine time constraint is binding:
`2*15 + 4*17.5 = 100`. The labor constraint is not fully used (`3*15 + 2*17.5 = 80 <= 90`).
This indicates machine hours are the limiting resource for this profit-maximizing plan.

## Insights & Extensions

**Business insights**
- Producing fractional units is allowed by the continuous LP formulation; if units must be integer, convert to an Integer Programming problem (use `cat="Integer"` in PuLP variables). The integer solution may differ slightly.
- The optimal mix prioritizes the product combination that uses machine hours efficiently because machine time is the bottleneck.
- If management can increase machine hours (e.g., overtime, additional shifts), the marginal benefit can be evaluated by sensitivity analysis (dual prices/shadow prices).

**Possible extensions**
- Add fixed costs or setup times (introduces binary variables -> MILP).
- Introduce demand upper bounds per product.
- Run sensitivity analysis on profit coefficients, resource capacities (shadow prices).
- Re-run with integer constraints to produce implementable production plans.